# Introduction

Our business problem began by identifying the most common products bought together and for that we did several recommendation systems.

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('orders (1).xls')
df.drop(columns="Unnamed: 0", axis =1, inplace=True)
df.head()

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id
0,1128254,6346669,2023-01-06,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,600.0,Não,AMB,Bebidas,Aguas Minerais,1
1,1001096,6001131,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2
2,1001096,6001131,2023-01-17,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,3
3,1121833,6358142,2023-01-04,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,12.0,Não,AMB,Bebidas,Aguas Minerais,4
4,1122758,6328488,2023-01-04,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,60.0,Não,AMB,Bebidas,Aguas Minerais,5


How the company works is that for every deivery place in a day a truck takes the products, there can be more than one delivery place per day, as well several clients can have the same delivery place

In [3]:
#Implementing a orders Id
#order = df.groupby(['date','delivery_place']).size().reset_index()
#order['order_id'] = [i+1 for i in range(len(order))]
#order = order.drop(0,axis=1)
#data=df.merge(order, on=['date','delivery_place'])
#data.head()

In [4]:
#make groupby to get indexes product_tyoe and subtype
analise = df.groupby(['warehouse_zone','product_type','product_subtype'])['quantity'].sum().reset_index()
analise2 = df.groupby(['warehouse_zone','product_type',])['quantity'].sum().reset_index()

#Create Id's 
analise['cellule'] = (analise.groupby('product_type').cumcount()+1).apply(lambda x: str(x).zfill(2))
analise2['ID'] = (analise2.groupby('warehouse_zone').cumcount()+1).apply(lambda x: str(x).zfill(2))
analise2['alley']= analise2.apply(lambda row: row['warehouse_zone'] +'-'+ str(row['ID']), axis=1)

#Drop unecessary columns
analise2=analise2.drop(['quantity','ID'], axis=1)
analise=analise.drop(['quantity'],axis=1)

#Merge everything together on a dataframe
df_locations = df.merge(analise2, on=['warehouse_zone','product_type'], how='left')
df_new = df_locations.merge(analise, on=['warehouse_zone','product_type','product_subtype'], how='left')
df_new['alleycell'] = df_new.apply(lambda row: row['alley'] +'-'+ str(row['cellule']), axis=1)
df_new.head(3)

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id,alley,cellule,alleycell
0,1128254,6346669,2023-01-06,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,600.0,Não,AMB,Bebidas,Aguas Minerais,1,AMB-01,02,AMB-01-02
1,1001096,6001131,2023-01-03,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2,AMB-01,02,AMB-01-02
2,1001096,6001131,2023-01-17,126898,AGUA DAS PEDRAS SALGADAS 6x0.33 PET,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,3,AMB-01,02,AMB-01-02


In [5]:
#remove frozenset
#df_new['product_description'] = df_new['product_description'].apply(lambda x: str(set(x)))

#df_new

df_new['product_description'] = df_new['product_subtype'].apply(lambda x: str(x).replace("{", "").replace("}", "").replace("'", ""))

df_new['product_product_type'] = df_new['product_type'].apply(lambda x: str(x).replace("{", "").replace("}", "").replace("'", ""))

df_new

,client,delivery_place,date,product,product_description,measure,quantity,is_internal_client,warehouse_zone,product_type,product_subtype,order_id,alley,cellule,alleycell,product_product_type
0,1128254,6346669,2023-01-06,126898,Aguas Minerais,UN,600.0,Não,AMB,Bebidas,Aguas Minerais,1,AMB-01,02,AMB-01-02,Bebidas
1,1001096,6001131,2023-01-03,126898,Aguas Minerais,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,2,AMB-01,02,AMB-01-02,Bebidas
2,1001096,6001131,2023-01-17,126898,Aguas Minerais,UN,120.0,Não,AMB,Bebidas,Aguas Minerais,3,AMB-01,02,AMB-01-02,Bebidas
3,1121833,6358142,2023-01-04,126898,Aguas Minerais,UN,12.0,Não,AMB,Bebidas,Aguas Minerais,4,AMB-01,02,AMB-01-02,Bebidas
4,1122758,6328488,2023-01-04,126898,Aguas Minerais,UN,60.0,Não,AMB,Bebidas,Aguas Minerais,5,AMB-01,02,AMB-01-02,Bebidas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434415,1002555,6010385,2023-01-31,623226,Fardamento,UN,2.0,Sim,NAL,Nao Alimentares,Fardamento,4690,NAL-06,34,NAL-06-34,Nao Alimentares
434416,1002555,6010385,2023-01-31,623260,Fardamento,UN,2.0,Sim,NAL,Nao Alimentares,Fardamento,4690,NAL-06,34,NAL-06-34,Nao Alimentares
434417,1002555,6320067,2023-01-31,126440,Peixe Congelado,KG,6.0,Sim,CON,Peixe,Peixe Congelado,14701,CON-09,11,CON-09-11,Peixe
434418,1002554,6107653,2023-01-31,563492,Massas Secas,KG,2.0,Sim,AMB,Mercearia,Massas Secas,14710,AMB-10,23,AMB-10-23,Mercearia


In [6]:
coords = pd.read_csv("coords.xls", delimiter=';')
coords['x'] = coords['x'].str.replace(',', '.').astype(float)
coords['y'] = coords['y'].str.replace(',', '.').astype(float)

gg = df_new.groupby(['warehouse_zone','product_type','alley']).size().reset_index()
s= coords.merge(gg, on=['warehouse_zone','product_type'], how='left')
s= s.drop(0,axis=1)
s.loc[len(s)] = ['START',"START", 15, 1,"START"]
s.tail()

,warehouse_zone,product_type,x,y,alley
51,REF,Peixe,5.5,34.0,REF-09
52,REF,Refeições Cook & Chill (Socigeste),5.5,32.0,REF-10
53,SAL,Legumes,27.0,25.5,SAL-01
54,SAL,Mercearia,27.0,20.5,SAL-02
55,START,START,15.0,1.0,START


## Recommendation system

## Apriori

### Association Rules

With the order id in place we can now start building a recommendation system based on what items are frequently purchased together. This can be useful for suggesting related items to customers, as well as informing product placement and marketing strategies.

To do this, we use the Apriori algorithm to identify frequent itemsets - that is, sets of items that appear together in a minimum number of orders. From these itemsets, we can generate association rules that tell us which items tend to be purchased together.

The Apriori algorithm is a scalable and efficient way to mine large datasets for frequent itemsets and generate association rules. By using this approach, we can identify patterns and relationships in transactional data that can inform our recommendation system and ultimately help us make data-driven decisions about product recommendations, marketing strategies, and product placement.

In [7]:
from mlxtend.frequent_patterns import apriori, association_rules

# Filter for orders with more than one item
df = df.groupby('order_id').filter(lambda x: len(x) > 1)

# Pivot data to create binary matrix
basket = pd.pivot_table(df, index='order_id', columns='product_subtype', values='quantity', aggfunc='sum', fill_value=0)

# Convert values to binary
basket[basket > 0] = 1

# Find frequent itemsets
freq_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

freq_itemsets.head()

/Users/hugoribeiro/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.174519,(Acucar - Adocantes)
1,0.138714,(Aguas Minerais)
2,0.078038,(Aperitivos)
3,0.275317,(Arroz)
4,0.294463,(Azeites)


In the context of the Apriori algorithm, support refers to the frequency with which an itemset appears in the dataset. Specifically, the support of an itemset is defined as the proportion of transactions in the dataset that contain all the items in that itemset.

For example, in the table you provided, the first itemset (`ABOBORA`) has a support value of `0.062928`. This means that about `6.3%` of the transactions in the dataset contain the item `ABOBORA`. Similarly, the second itemset (`ACUCAR GRANULADO SACO PAPEL 1 KG RAR`) has a support value of `0.135336`, which means that about `13.5%` of the transactions in the dataset contain the item `ACUCAR GRANULADO SACO PAPEL 1 KG RAR`.

The support metric is important in the Apriori algorithm because it is used to identify frequent itemsets, which are then used to generate association rules. Specifically, itemsets with a support value above a given threshold (e.g., `0.05` or `0.1`) are considered frequent, and the algorithm generates association rules based on those itemsets.

In [8]:
# Generate association rules
rules_original = association_rules(freq_itemsets, metric='lift', min_threshold=1)

# Sort rules by lift and support
rules_original = rules_original.sort_values(['lift', 'support'], ascending=[False, False])

# Print top 5 rules
print(rules_original.shape)
rules_original.head()

(200628, 10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4676,"(Sacos de Plastico, Biosog)",(Panos),0.076678,0.061145,0.05115,0.667075,10.909717,0.046461,2.820017,0.983772
4681,(Panos),"(Sacos de Plastico, Biosog)",0.061145,0.076678,0.05115,0.836531,10.909717,0.046461,5.648305,0.967496
4678,"(Panos, Biosog)",(Sacos de Plastico),0.052792,0.102769,0.05115,0.968889,9.427864,0.045724,28.839579,0.943754
4679,(Sacos de Plastico),"(Panos, Biosog)",0.102769,0.052792,0.05115,0.497717,9.427864,0.045724,1.885805,0.996322
4680,(Biosog),"(Sacos de Plastico, Panos)",0.100422,0.054106,0.05115,0.509346,9.413841,0.045716,1.927822,0.993548


In [9]:
rules_orig2 = rules_original[rules_original["antecedents"] == "ALFACE"]

rules_orig2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


- Metrics:
    - antecedents:
        - the antecedent itemset of the association rule
    - consequents: 
        - the consequent itemset of the association rule
    - antecedent support: 
        - the support of the antecedent itemset, i.e., the proportion of transactions that contain all the items in the antecedent
    - consequent support: 
        - the support of the consequent itemset, i.e., the proportion of transactions that contain all the items in the consequent
    - support:
        - the support of the rule, i.e., the proportion of transactions that contain both the antecedent and the consequent
    - confidence: 
        - the confidence of the rule, i.e., the proportion of transactions that contain the consequent given that they also contain the antecedent
    - lift: 
        - the lift of the rule, which measures how much more often the antecedent and consequent co-occur in the dataset than we would expect if they were independent. A lift greater than 1 indicates a positive correlation between the antecedent and consequent, while a lift less than 1 indicates a negative correlation, and a lift equal to 1 indicates independence.
    - leverage: 
        - the leverage of the rule, which measures the difference between the observed frequency of the antecedent and consequent co-occurring and the frequency that would be expected if they were independent. A leverage of 0 indicates independence, while a positive leverage indicates a positive correlation, and a negative leverage indicates a negative correlation.
    - conviction: 
        - the conviction of the rule, which measures how much the antecedent and consequent are dependent on each other. A conviction value greater than 1 indicates that the antecedent and consequent are positively dependent, while a conviction value less than 1 indicates that they are negatively dependent, and a conviction value equal to 1 indicates independence.
    - zhang's metric: 
        - a metric that combines lift and conviction to give an overall measure of the interestingness of the rule. A value greater than 0 indicates that the rule is interesting.
        
The most important metrics for evaluating association rules are usually **support, confidence, and lift**. Support and confidence are used to filter out rules that are not frequent or not strong enough, while lift is used to identify rules that represent interesting correlations between items. In general, a high support value indicates that the rule is frequent, a high confidence value indicates that the rule is strong, and a high lift value indicates that the rule is interesting.

In [10]:
rules_original = rules_original[(rules_original['confidence']>=0.7) & (rules_original['support']>= 0.05)]

rules_original

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4681,(Panos),"(Sacos de Plastico, Biosog)",0.061145,0.076678,0.051150,0.836531,10.909717,0.046461,5.648305,0.967496
4678,"(Panos, Biosog)",(Sacos de Plastico),0.052792,0.102769,0.051150,0.968889,9.427864,0.045724,28.839579,0.943754
4677,"(Sacos de Plastico, Panos)",(Biosog),0.054106,0.100422,0.051150,0.945360,9.413841,0.045716,16.463699,0.944898
4684,"(Biosog, Utensilios Domesticos)",(Sacos de Plastico),0.065321,0.102769,0.062224,0.952586,9.269229,0.055511,18.923425,0.954463
886,(Panos),(Utensilios Domesticos),0.061145,0.089207,0.050540,0.826554,9.265580,0.045085,5.251165,0.950172
...,...,...,...,...,...,...,...,...,...,...
85224,"(Horticolas Frescos, Carne Porco Congelada, Fr...",(Leite),0.097044,0.362412,0.067996,0.700677,1.933371,0.032826,2.130100,0.534653
4737,"(Bovino Adulto Congelado (Nacional), Horticola...",(Leite),0.075551,0.362412,0.052933,0.700621,1.933217,0.025552,2.129703,0.522179
28657,"(Peixe Congelado, Carne Porco Congelada, Horti...",(Leite),0.094510,0.362412,0.066213,0.700596,1.933147,0.031962,2.129523,0.533091
30855,"(Horticolas Frescos, Criacao (Ovos), Frutas Fr...",(Leite),0.071703,0.362412,0.050211,0.700262,1.932226,0.024225,2.127149,0.519728


Explicar

## Q-Learn

In [11]:
rules_final = (rules_original['antecedents'].apply(lambda x: list(x)) + rules_original['consequents'].apply(lambda x: list(x))).reset_index()
rules_final.rename(columns = {'index':'order_id',0:'items'}, inplace = True)

rules_final

,order_id,items
0,4681,"[Panos, Sacos de Plastico, Biosog]"
1,4678,"[Panos, Biosog, Sacos de Plastico]"
2,4677,"[Sacos de Plastico, Panos, Biosog]"
3,4684,"[Biosog, Utensilios Domesticos, Sacos de Plast..."
4,886,"[Panos, Utensilios Domesticos]"
...,...,...
45554,85224,"[Horticolas Frescos, Carne Porco Congelada, Fr..."
45555,4737,"[Bovino Adulto Congelado (Nacional), Horticola..."
45556,28657,"[Peixe Congelado, Carne Porco Congelada, Horti..."
45557,30855,"[Horticolas Frescos, Criacao (Ovos), Frutas Fr..."


In [12]:
rules_final2 = rules_final.copy()

rules_final2 = rules_final[["order_id", "items"]]

rules_final2

,order_id,items
0,4681,"[Panos, Sacos de Plastico, Biosog]"
1,4678,"[Panos, Biosog, Sacos de Plastico]"
2,4677,"[Sacos de Plastico, Panos, Biosog]"
3,4684,"[Biosog, Utensilios Domesticos, Sacos de Plast..."
4,886,"[Panos, Utensilios Domesticos]"
...,...,...
45554,85224,"[Horticolas Frescos, Carne Porco Congelada, Fr..."
45555,4737,"[Bovino Adulto Congelado (Nacional), Horticola..."
45556,28657,"[Peixe Congelado, Carne Porco Congelada, Horti..."
45557,30855,"[Horticolas Frescos, Criacao (Ovos), Frutas Fr..."


In [13]:
# explode the items column
rules_final3 = rules_final2.explode('items')

# merge the order_id column with the exploded dataframe
rules_final3 = pd.merge(rules_final2, rules_final2[['order_id']], left_index=True, right_index=True)

# rename the order_id column to item_order_id
rules_final3 = rules_final2.rename(columns={'order_id_x': 'order_id', 'order_id_y': 'item_order_id'})

rules_final3 = rules_final2[["order_id", "items"]]

rules_final3 = rules_final2.rename(columns = {"items": "product_subtype"})

# display the resulting dataframe
print(rules_final3)

       order_id                                    product_subtype
0          4681                 [Panos, Sacos de Plastico, Biosog]
1          4678                 [Panos, Biosog, Sacos de Plastico]
2          4677                 [Sacos de Plastico, Panos, Biosog]
3          4684  [Biosog, Utensilios Domesticos, Sacos de Plast...
4           886                     [Panos, Utensilios Domesticos]
...         ...                                                ...
45554     85224  [Horticolas Frescos, Carne Porco Congelada, Fr...
45555      4737  [Bovino Adulto Congelado (Nacional), Horticola...
45556     28657  [Peixe Congelado, Carne Porco Congelada, Horti...
45557     30855  [Horticolas Frescos, Criacao (Ovos), Frutas Fr...
45558      4821  [Criacao Congelada, Carne Porco Congelada, Leite]

[45559 rows x 2 columns]


In [14]:
test_new = df_new.copy()
# df is your original dataframe
test_new = test_new [['product_type', 'product_subtype']] # select only the columns you want to keep
test_new

,product_type,product_subtype
0,Bebidas,Aguas Minerais
1,Bebidas,Aguas Minerais
2,Bebidas,Aguas Minerais
3,Bebidas,Aguas Minerais
4,Bebidas,Aguas Minerais
...,...,...
434415,Nao Alimentares,Fardamento
434416,Nao Alimentares,Fardamento
434417,Peixe,Peixe Congelado
434418,Mercearia,Massas Secas


In [15]:
rules_final3

# use explode to separate the list of items and give them the same order ID
rules_final4 = rules_final3.explode('product_subtype')

# reset the index and drop the old index
rules_final4 = rules_final4.reset_index(drop=True)

print(rules_final4)

        order_id        product_subtype
0           4681                  Panos
1           4681      Sacos de Plastico
2           4681                 Biosog
3           4678                  Panos
4           4678                 Biosog
...          ...                    ...
249925     30855         Frutas Frescas
249926     30855                  Leite
249927      4821      Criacao Congelada
249928      4821  Carne Porco Congelada
249929      4821                  Leite

[249930 rows x 2 columns]


In [ ]:
# Convert data types
test_new['product_type'] = test_new['product_type'].astype('category')
test_new['product_subtype'] = test_new['product_subtype'].astype('category')

# Merge data frames
merged_df = pd.merge(rules_final4, test_new, on='product_subtype', how='left', suffixes=('_rules', '_test'))

# Convert data types back to original
merged_df['product_type'] = merged_df['product_type'].astype(test_new['product_type'].dtype)
merged_df['product_subtype'] = merged_df['product_subtype'].astype(test_new['product_subtype'].dtype)

merged_df.head(5)

In [ ]:
#merged_df = pd.merge(rules_final4, test_new[['product_subtype', 'product_type']], on='product_subtype', how='left')
#test_eu=merged_df.drop_duplicates()


#merged_df.head(50)

#test_eu.head(5)

In [ ]:
grouped_df = merged_df.groupby(['order_id', 'product_type']).agg({'product_subtype': 'first'}).reset_index()

print(grouped_df)


In [ ]:
rules_final5 = test_eu.copy()

rules_final5

In [ ]:
s_new = s.copy()

s_new

In [ ]:
s_new['product_type'] = s_new['product_type'].apply(lambda x: str(x).replace("{", "").replace("}", "").replace("'", ""))

s_new

In [ ]:
# Create a dictionary mapping product_type to warehouse_zone, x, y, and alley
prod_type_dict = s_new.set_index('product_type')[['warehouse_zone', 'x', 'y', 'alley']].to_dict()

# Use the map method to create new columns in rules_final
rules_final5['warehouse_zone'] = rules_final5['product_type'].map(prod_type_dict['warehouse_zone'])
rules_final5['x'] = rules_final5['product_type'].map(prod_type_dict['x'])
rules_final5['y'] = rules_final5['product_type'].map(prod_type_dict['y'])
rules_final5['alley'] = rules_final5['product_type'].map(prod_type_dict['alley'])

rules_final5

In [ ]:
rules_final6 = rules_final5.copy()

In [ ]:
#mask = rules_final6.duplicated(subset=['order_id'], keep=False)

#duplicated_rows = rules_final6.loc[mask]

#print(duplicated_rows)


In [ ]:
rules_final_920 = rules_final5[rules_final5['order_id'] == 920]

rules_final_920

In [ ]:
rules_final69 = rules_final6[rules_final6['order_id'] == 35]

rules_final69

In [ ]:
import numpy as np
import pandas as pd
import time
import math

start_time = time.time()

# Function for calculating distance between zones
def calculate_distance(zone1, zone2):
    zone1_df = rules_final_920[rules_final_920["alley"] == zone1]
    if zone1_df.empty:
        return float('inf')
    x1, y1 = float(zone1_df["x"].iloc[0]), float(zone1_df["y"].iloc[0])
    x2, y2 = float(rules_final_920[rules_final_920["alley"] == zone2]["x"].iloc[0]), float(rules_final_920[rules_final_920["alley"] == zone2]["y"].iloc[0])
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance


def get_shortest_distance_zone(prev_zone, zone_list):
    distances = []
    for zone in zone_list:
        if zone == prev_zone:
            distances.append(float('inf'))
        else:
            distances.append(calculate_distance(prev_zone, zone))
    return zone_list[np.argmin(distances)]


# Parameters for Q-learning
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1# Exploration rate
n_episodes = 10 #1000  # Number of episodes

# Get the unique order ids and zones
order_ids = rules_final_920['order_id'].unique()
zones = rules_final_920['alley'].unique()
min_order_id = np.min(order_ids)

# Initialize the Q-table
num_zones = len(order_ids)
q_table = np.zeros((len(order_ids), len(zones), len(zones)))

for episode in range(n_episodes):
    for order_id in order_ids:
        order = rules_final_920[rules_final_920['order_id'] == order_id]
        zone_list = order['alley'].tolist()

        for i in range(len(zone_list) - 1):
            current_zone = zone_list[i]
            next_zone = zone_list[i + 1]

            current_zone_idx = np.where(zones == current_zone)[0][0]
            next_zone_idx = np.where(zones == next_zone)[0][0]

            # Choose action (next zone) using epsilon-greedy strategy
            if np.random.random() < epsilon:
                action_idx = np.random.randint(num_zones)
            else:
                action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :])

            # Calculate the reward based on the distance to the chosen zone
            reward = -calculate_distance(current_zone, zones[min(action_idx, len(zones)-1)])

            # Update Q-table
            q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)] = q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)] + alpha * (reward + gamma * np.max(q_table[order_id - min_order_id, min(action_idx, len(zones)-1), :]) - q_table[order_id - min_order_id, current_zone_idx, min(action_idx, len(zones)-1)])

# Print the optimal picking order
for order_id in order_ids:
    order = rules_final_920[rules_final_920['order_id'] == order_id]
    zone_list = order['alley'].tolist()

    first_zone = get_shortest_distance_zone("START", zone_list)
    optimal_order = [first_zone]

    for i in range(len(zone_list) - 1):
        current_zone = zone_list[i + 1]
        current_zone_idx = np.where(zones == current_zone)[0][0]

        # Choose the best action based on the Q-table
        action_idx = np.argmax(q_table[order_id - min_order_id, current_zone_idx, :])

        next_zone = zones[action_idx]
        next_zone = get_shortest_distance_zone(optimal_order[-1], [next_zone] + zone_list[i + 2:])
        if next_zone not in optimal_order:
            optimal_order.append(next_zone)

    print(f"Optimal picking order for order {order_id}: {optimal_order}")


end_time = time.time()

print("Running time:", end_time - start_time, "seconds")